In [ ]:
def create_csv(results,Dinkpc,signal_name,results_directory):
  values = {
          'beta_est': result.posterior['beta'].mean(),
          'beta_un': result.posterior['beta'].std(),
          'alpha_est': result.posterior['alpha'].mean(),
          'alpha_un': result.posterior['alpha'].std(),
          'tau_est': result.posterior['tau'].mean(),
          'tau_un': result.posterior['tau'].std(),
          's_est': result.posterior['s'].mean(),
          's_un': result.posterior['s'].std(),
      }
  df = pd.DataFrame.from_dict(values, orient='index')
  final = df.T
  final.to_csv(results_directory + f'{signal_name}.csv')

In [ ]:
def merge_csv(n,Dinkpc,signal_name,results_directory):
  rows = []

  # Lee el encabezado del primer archivo
  primer_df = pd.read_csv(f"{Dinkpc}{signal_name}.csv")
  columnas_numericas = primer_df.select_dtypes(include='number').columns.tolist()

  # Añadimos una columna extra para el nombre del archivo
  columnas_finales = ["archivo"] + columnas_numericas

  for k in range(n):
      # Lee cada archivo
      df = pd.read_csv(f"{Dinkpc}signal_{k}.csv")

    # Extrae los valores numéricos
      valores = df.select_dtypes(include='number').values.flatten()

    # Crea una fila con el nombre del archivo + los datos
      fila = [f"1signal_{k}"] + list(valores)

      rows.append(fila)

  # Crea el DataFrame final con encabezado del primer archivo
  final_df = pd.DataFrame(rows, columns=columnas_finales)

  # Guarda el resultado
  output_path = os.path.join(results_directory, results_directory + "_merged.csv")
  final_df.to_csv(output_path, index=False)


In [ ]:
def read_n_plot_results(file,figs_dir,Dinkpc):
  resultados = pd.read_csv(file)
  abyl = pd.read_csv('/Abylkairov_catalog.csv')  #Load catalog
  ab_0 = abyl[(abyl['t(ms)'] == 0) & (abyl['GR_or_GREP'] == 0)]  #Take GR waveforms and the values of the parameters at bounce t = 0

  # Change the names of the EOS and add a column
  eos_map = {
      0: 'SFHo',
      1: 'LS220',
      2: 'HSDD2',
      3: 'GShenFSU2.1'
  }

  ab_0 = ab_0.copy()

  ab_0['eos_name'] = ab_0['EOS'].map(eos_map)

  resultados_eos = resultados.copy()
  resultados_eos['eos_name'] = ab_0['eos_name'].reset_index(drop=True)
  resultados_eos['T/|W|'] = ab_0['T/|W|'].reset_index(drop=True)
  resultados_eos['fpeak'] = ab_0['f_peak'].reset_index(drop=True)
  resultados_eos['Deltah'] = ab_0['D Delta h'].reset_index(drop=True)
  display(resultados_eos.head())

  #Estimation quality
  sns.scatterplot(data = resultados_eos,x='T/|W|',y='beta_est', hue='eos_name')
  plt.plot([0.02,0.18],[0.02,0.18],color='tab:gray',linestyle='--',label=r'$\hat{\beta} = \beta$')
  plt.grid(True)
  plt.xlabel(r'$\beta$',fontsize=14)
  plt.ylabel(r'$\hat{\beta}$',fontsize=14)
  plt.title(f'$\beta$ estimation @ {Dinkpc}', fontsize = 16)
  plt.legend(title='Equation of State')
  plt.savefig(figs_dir + f'beta_est_{Dinkpc}.png')

  sns.scatterplot(data = resultados_eos,x='beta_est',y='alpha_est', hue='eos_name')
  plt.grid(True)
  plt.title(fr'$\beta$ vs $\alpha$ @ {Dinkpc}', fontsize=16)
  plt.xlabel(r'$\beta$',fontsize=14)
  plt.ylabel(r'$\alpha$',fontsize=14)
  plt.legend(title='Equation of State')
  plt.savefig(figs_dir + fr'beta_vs_alpha_{Dinkpc}.png')

  return resultados_eos
